In [17]:
"""
Analysis of future fire probability for the Southern Rockies

Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import geopandas as gpd

from sklearn.linear_model import TheilSenRegressor
from tqdm.notebook import tqdm

# Custom functions
sys.path.append(os.path.join(maindir,'aspen-fire/Aim2/code/Python'))
from __functions import *

# Environment variables
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/')

proj = 'EPSG:5070' # albers equal area

print("Ready to go !")

Ready to go !


## Future Fire

From "Fires of Unusual Size", load and process the annual projected fire activity (area burned and fire counts). Process the grid outputs (geospatial vector format) which can serve as the analystical unit for the remainder of the analysis. Subset grids to Southern Rockies Level III ecoregion. Once merged, tidy the dataframe and set up the median-based linear model (MBLM) to retrieve the trend in area burned and fire counts.

In [2]:
# load the future fire probability annual data
fp = os.path.join(projdir,'Aim3/data/spatial/raw/future-fire/CNRM/')
grids = list_files(fp, '*.shp', recursive = True)
print(f"Total grids: {len(grids)}\n")

# Read the files into a list
gdfs = []
with tqdm(grids, total=len(grids)) as pbar:
    for grid in pbar:
        pbar.set_description(f"Processing [{os.path.basename(grid)[:-4]}]")
        gdf = gpd.read_file(grid)
        gdf = gdf[gdf['NA_L3NAME'] == 'Southern Rockies']
        gdfs.append(gdf)

# grab the crs
crs = gdfs[0].crs

# Merge all into a single GeoDataFrame
future_fire = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=crs)
future_fire.columns

Total grids: 37



  0%|          | 0/37 [00:00<?, ?it/s]

Index(['values', 'grid_id', 'US_L4NAME', 'US_L4CODE', 'US_L3CODE', 'US_L3NAME',
       'NA_L3CODE', 'NA_L3NAME', 'NA_L2CODE', 'NA_L2NAME', 'NA_L1CODE',
       'NA_L1NAME', 'L4_KEY', 'L3_KEY', 'L2_KEY', 'L1_KEY', 'Shape_Leng',
       'Shape_Area', 'Year', 'En_Area', 'En_NFire', 'geometry'],
      dtype='object')

In [4]:
future_fire.head()

,values,grid_id,US_L4NAME,US_L4CODE,US_L3CODE,US_L3NAME,NA_L3CODE,NA_L3NAME,NA_L2CODE,NA_L2NAME,...,L4_KEY,L3_KEY,L2_KEY,L1_KEY,Shape_Leng,Shape_Area,Year,En_Area,En_NFire,geometry
0,0.0,159230.0,Foothill Shrublands,21d,21,Southern Rockies,6.2.14,Southern Rockies,6.2,WESTERN CORDILLERA,...,21d Foothill Shrublands,21 Southern Rockies,6.2 WESTERN CORDILLERA,6 NORTHWESTERN FORESTED MOUNTAINS,1.597635e+06,7.271793e+09,2049.0,22744.632243,2.055556,"POLYGON ((-106.61317 42.78435, -106.57317 42.7..."
1,0.0,159231.0,Foothill Shrublands,21d,21,Southern Rockies,6.2.14,Southern Rockies,6.2,WESTERN CORDILLERA,...,21d Foothill Shrublands,21 Southern Rockies,6.2 WESTERN CORDILLERA,6 NORTHWESTERN FORESTED MOUNTAINS,1.597635e+06,7.271793e+09,2049.0,22744.632243,2.055556,"POLYGON ((-106.57317 42.78435, -106.53317 42.7..."
2,0.0,159232.0,Foothill Shrublands,21d,21,Southern Rockies,6.2.14,Southern Rockies,6.2,WESTERN CORDILLERA,...,21d Foothill Shrublands,21 Southern Rockies,6.2 WESTERN CORDILLERA,6 NORTHWESTERN FORESTED MOUNTAINS,1.597635e+06,7.271793e+09,2049.0,22744.632243,2.055556,"POLYGON ((-106.53317 42.78435, -106.49317 42.7..."
3,0.0,159233.0,Crystalline Mid-Elevation Forests,21c,21,Southern Rockies,6.2.14,Southern Rockies,6.2,WESTERN CORDILLERA,...,21c Crystalline Mid-Elevation Forests,21 Southern Rockies,6.2 WESTERN CORDILLERA,6 NORTHWESTERN FORESTED MOUNTAINS,6.022792e+04,1.441915e+08,2049.0,84116.649792,4.498282,"POLYGON ((-106.49317 42.78435, -106.45317 42.7..."
4,0.0,159233.0,Foothill Shrublands,21d,21,Southern Rockies,6.2.14,Southern Rockies,6.2,WESTERN CORDILLERA,...,21d Foothill Shrublands,21 Southern Rockies,6.2 WESTERN CORDILLERA,6 NORTHWESTERN FORESTED MOUNTAINS,1.597635e+06,7.271793e+09,2049.0,22744.632243,2.055556,"POLYGON ((-106.49317 42.78435, -106.45317 42.7..."


In [6]:
future_fire['US_L4NAME'].unique()

array(['Foothill Shrublands', 'Crystalline Mid-Elevation Forests',
       'Crystalline Subalpine Forests', 'Alpine Zone',
       'Sedimentary Mid-Elevation Forests',
       'Sedimentary Subalpine Forests', 'Sagebrush Parks',
       'Volcanic Subalpine Forests', 'Grassland Parks',
       'Volcanic Mid-Elevation Forests'], dtype=object)

In [7]:
future_fire.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
future_fire = future_fire.to_crs(proj) # reproject
future_fire.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [10]:
future_fire_ = future_fire[['grid_id','NA_L3NAME','US_L4NAME','US_L4CODE','Year','En_Area','En_NFire','geometry']]
future_fire_['grid_id'] = future_fire_.grid_id.astype(int)
future_fire_['Year'] = future_fire_.Year.astype(int)
future_fire_.head()

,grid_id,NA_L3NAME,US_L4NAME,US_L4CODE,Year,En_Area,En_NFire,geometry
0,159230,Southern Rockies,Foothill Shrublands,21d,2049,22744.632243,2.055556,"POLYGON ((-861518.632 2246765.246, -858285.087..."
1,159231,Southern Rockies,Foothill Shrublands,21d,2049,22744.632243,2.055556,"POLYGON ((-858285.087 2246403.307, -855051.389..."
2,159232,Southern Rockies,Foothill Shrublands,21d,2049,22744.632243,2.055556,"POLYGON ((-855051.389 2246042.730, -851817.539..."
3,159233,Southern Rockies,Crystalline Mid-Elevation Forests,21c,2049,84116.649792,4.498282,"POLYGON ((-851817.539 2245683.513, -848583.540..."
4,159233,Southern Rockies,Foothill Shrublands,21d,2049,22744.632243,2.055556,"POLYGON ((-851817.539 2245683.513, -848583.540..."


In [14]:
print(future_fire_['Year'].min())
print(future_fire_['Year'].max())

2023
2060


In [18]:
future_fire_[future_fire_['US_L4CODE'] == '21d']['En_NFire'].describe()

count    117387.000000
mean          1.187936
std           0.521148
min           0.513173
25%           0.743414
50%           1.085911
75%           1.595074
max           2.540664
Name: En_NFire, dtype: float64

In [32]:
# save a spatial dataframe (no duplicates)
fire_grid = future_fire_.drop_duplicates(subset=['grid_id','NA_L3NAME','US_L4NAME','US_L4CODE'])
fire_grid = fire_grid[['grid_id','NA_L3NAME','US_L4NAME','US_L4CODE','geometry']]
# Export this spatial dataframe
out_fp = os.path.join(projdir,'Aim3/data/spatial/mod/future_fire_grid.gpkg')
fire_grid.to_file(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/spatial/mod/future_fire_grid.gpkg


In [21]:
# create a grouped time-series by ecoregion
future_fire_eco = (
    future_fire_.groupby(['US_L4CODE','Year'], as_index=False)
    .agg(
        En_Area=('En_Area','mean'),
        En_NFire=('En_NFire','mean'),
    )
)
future_fire_eco.head()

,US_L4CODE,Year,En_Area,En_NFire
0,21a,2023,740.474306,0.073310
1,21a,2024,560.702556,0.054410
2,21a,2025,2061.189727,0.161512
3,21a,2026,816.846677,0.092784
4,21a,2027,4828.760898,0.309851


In [23]:
future_fire_eco[(future_fire_eco['US_L4CODE'] == '21d') & (future_fire_eco['Year'] == 2049)]

,US_L4CODE,Year,En_Area,En_NFire
137,21d,2049,22744.632243,2.055556


In [24]:
# check for duplicates, remove them
n = future_fire_eco.duplicated(subset=['US_L4CODE','Year']).sum()
if n > 0:
    print(f"\nThere are [{n}] duplicate rows.\n")
else:
    print("\nNo duplicates at this stage.\n")


No duplicates at this stage.



In [26]:
# save this table out for modeling in R
out_fp = os.path.join(projdir,'Aim3/data/tabular/us_l4eco_future-fire.csv')
future_fire_eco.to_csv(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/tabular/us_l4eco_future-fire.csv


gc.collect()